In [49]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import optuna as op
from sklearn.model_selection import train_test_split as tts
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler as ss

In [ ]:
data = pd.read_csv("diabetes.csv")
data.dropna(inplace=True)
X = data.to_numpy()[:,0:8]
y = data.to_numpy()[:,8]

X = ss().fit_transform(X)

X = PCA(n_components=3, random_state=86).fit_transform(X)

X_train, X_test, y_train, y_test = tts(X, y, train_size=0.8, random_state=86)

model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(X.shape[1],))
    ]
)

model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.1), loss='huber', metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=200, validation_data=[X_test, y_test])

loss, accuracy = model.evaluate(X_test, y_test)

print(f'Loss: {loss}\nAccuracy: {accuracy}')

Epoch 1/200


/SI/si/lib/python3.11/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.7013 - loss: 0.0952 - val_accuracy: 0.7273 - val_loss: 0.0943
Epoch 2/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6991 - loss: 0.0969 - val_accuracy: 0.7273 - val_loss: 0.0963
Epoch 3/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7094 - loss: 0.0977 - val_accuracy: 0.7013 - val_loss: 0.0921
Epoch 4/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7205 - loss: 0.0959 - val_accuracy: 0.7013 - val_loss: 0.0951
Epoch 5/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7307 - loss: 0.0901 - val_accuracy: 0.7078 - val_loss: 0.0944
Epoch 6/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7138 - loss: 0.0926 - val_accuracy: 0.7208 - val_loss: 0.0976
Epoch 7/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7473 - loss: 0.0845 - val_accuracy: 0.7143 - val_loss: 0.0865
Epoch 8/200
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7295 - loss: 0.0926 - val_accuracy: 0.7143 - val_loss: 0.

In [ ]:
X_train, X_test, y_train, y_test = tts(X, y, train_size=0.8, random_state=86)

def objective(trial):
    #l_r=0.1
    l_r = trial.suggest_categorical('learning_rate', [0.1, 0.01, 0.001, 0.001, 0.0001])
    optimizer = trial.suggest_categorical('optimizer', ['Adam', 'RMSprop', 'SGD', 'Adagrad'])
    loss = trial.suggest_categorical('loss', ['binary_crossentropy', 'categorical_crossentropy', 'sparse_categorical_crossentropy'])
    epochs = trial.suggest_int("epochs", 50, 300, 50)

    model = tf.keras.models.Sequential(
    [
        tf.keras.layers.Dense(1, activation='sigmoid', input_shape=(X.shape[1],))
    ]
)

    if optimizer=="Adam":
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=l_r), loss=loss, metrics=['accuracy'])
        history = model.fit(X_train, y_train, epochs=epochs, validation_data=[X_test, y_test])

    loss, accuracy = model.evaluate(X_test, y_test)
